In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier
import time

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

from cc3_preprocessor import Preprocessor

import joblib

np.random.seed(42)

In [2]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [3]:
pre = Preprocessor(smoothing_factor=5, mode = 'catboost')

In [4]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [5]:
y_train.value_counts(normalize=True)

subrogation
0.0    0.77141
1.0    0.22859
Name: proportion, dtype: float64

In [6]:
y_test.value_counts(normalize=True)

subrogation
0.0    0.771296
1.0    0.228704
Name: proportion, dtype: float64

In [7]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Fit complete.
Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.


In [8]:
print("Saving preprocessor and training columns...")

# Save the 'pre' object
joblib.dump(pre, 'cc3_preprocessor.pkl')

# Save the exact column order and names
joblib.dump(X_train_proc.columns, 'training_columns.pkl')

print("Done.")

Saving preprocessor and training columns...
Done.


## CatBoost with Optuna Tuning

In [9]:
import optuna
from optuna.integration import CatBoostPruningCallback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
CAT_FEATURES = pre.cat_for_encoding_
print(CAT_FEATURES)

['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season']


In [11]:
def objective(trial: optuna.trial.Trial) -> float:

    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 1.0, log=True), 
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        
        'eval_metric': 'F1',
        'task_type': 'CPU',
        'verbose': False,
        'early_stopping_rounds': 100,
        'random_state': 42
    }

    params['eval_metric'] = 'Logloss'
    
    model = CatBoostClassifier(**params)
    
    model.fit(
        X_train_proc, y_train,
        eval_set=(X_test_proc, y_test),
        cat_features=CAT_FEATURES,
        verbose=False
    )

    y_preds = model.predict(X_test_proc)
    
    manual_f1_score = f1_score(y_test, y_preds, pos_label=1)
    
    return manual_f1_score

In [12]:
print("\n2. Starting Optuna study...")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
)

study.optimize(
    objective, 
    n_trials=100, # Number of trials to run
    show_progress_bar=True
)

print("\n" + "="*50)
print("Optuna study finished.")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
    
print(f"  Value (Max F1 Score): {best_trial.value:.4f}") # <-- CHANGED comment
    
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-07 11:14:58,418] A new study created in memory with name: no-name-364a0fd2-32e5-4414-966c-3af07ef260b4



2. Starting Optuna study...


Best trial: 0. Best value: 0.54098:   1%|          | 1/100 [00:02<03:46,  2.29s/it]

[I 2025-11-07 11:15:00,716] Trial 0 finished with value: 0.540979571745016 and parameters: {'learning_rate': 0.02485482372534955, 'depth': 5, 'l2_leaf_reg': 0.02103275816497219, 'subsample': 0.9, 'random_strength': 0.019497543532053744, 'bagging_temperature': 0.30798298383762224, 'border_count': 250, 'scale_pos_weight': 6.23023418473191}. Best is trial 0 with value: 0.540979571745016.


Best trial: 0. Best value: 0.54098:   2%|▏         | 2/100 [00:05<04:36,  2.83s/it]

...

[I 2025-11-07 11:21:41,031] Trial 97 finished with value: 0.5276190476190477 and parameters: {'learning_rate': 0.04398026324175591, 'depth': 3, 'l2_leaf_reg': 0.1016081650051483, 'subsample': 1.0, 'random_strength': 1.1285856211410941e-05, 'bagging_temperature': 0.5611047303211386, 'border_count': 143, 'scale_pos_weight': 1.0305218555246203}. Best is trial 31 with value: 0.603374131657294.


Best trial: 31. Best value: 0.603374:  99%|█████████▉| 99/100 [06:44<00:01,  1.92s/it]

[I 2025-11-07 11:21:42,690] Trial 98 finished with value: 0.6021947873799726 and parameters: {'learning_rate': 0.0669330753109842, 'depth': 3, 'l2_leaf_reg': 0.37713955210755007, 'subsample': 1.0, 'random_strength': 3.4302947173046877e-06, 'bagging_temperature': 0.625954592886709, 'border_count': 164, 'scale_pos_weight': 2.321232732756329}. Best is trial 31 with value: 0.603374131657294.


Best trial: 31. Best value: 0.603374: 100%|██████████| 100/100 [06:45<00:00,  4.05s/it]

[I 2025-11-07 11:21:43,648] Trial 99 finished with value: 0.5450941526263627 and parameters: {'learning_rate': 0.09539964815175042, 'depth': 3, 'l2_leaf_reg': 0.19277899252389472, 'subsample': 1.0, 'random_strength': 2.8994150357351e-06, 'bagging_temperature': 0.6146467575776378, 'border_count': 164, 'scale_pos_weight': 6.052456353967993}. Best is trial 31 with value: 0.603374131657294.

Optuna study finished.
Number of finished trials: 100

Best trial:
  Value (Max F1 Score): 0.6034
  Best Hyperparameters:
    learning_rate: 0.029306960143738885
    depth: 8
    l2_leaf_reg: 2.160435718308443
    subsample: 0.9
    random_strength: 0.08003284959085531
    bagging_temperature: 0.08294170454932412
    border_count: 147
    scale_pos_weight: 2.8549710208640757


In [13]:
best_params = study.best_trial.params
print(best_params)

final_params = best_params.copy()
final_params.update({
    'iterations': 1000, # Use more iterations for the final model
    'eval_metric': 'Logloss', # Use Logloss for training/stopping
    'task_type': 'CPU',
    'early_stopping_rounds': 50 # Keep early stopping
})

best_model = CatBoostClassifier(**final_params)

best_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),
    cat_features=CAT_FEATURES,
    verbose=False
)

print(f"\nFinal Model Score (from best Logloss iteration):")
y_preds_final = best_model.predict(X_test_proc)
final_f1 = f1_score(y_test, y_preds_final, pos_label=1)
print(f"  Manual F1:class=1 Score: {final_f1:.4f}")
        
print("\n  Full Classification Report:")
print(classification_report(y_test, y_preds_final, target_names=['Class 0.0', 'Class 1.0']))

{'learning_rate': 0.029306960143738885, 'depth': 8, 'l2_leaf_reg': 2.160435718308443, 'subsample': 0.9, 'random_strength': 0.08003284959085531, 'bagging_temperature': 0.08294170454932412, 'border_count': 147, 'scale_pos_weight': 2.8549710208640757}

Final Model Score (from best Logloss iteration):
  Manual F1:class=1 Score: 0.5973

  Full Classification Report:
              precision    recall  f1-score   support

   Class 0.0       0.91      0.78      0.84      4165
   Class 1.0       0.50      0.74      0.60      1235

    accuracy                           0.77      5400
   macro avg       0.71      0.76      0.72      5400
weighted avg       0.82      0.77      0.78      5400



In [16]:
print("Saving best_model...")

# Use the model's F1 score in the name
best_model.save_model("catboost_mod_f1_0.6034.cbm")

print("Done.")

Saving best_model...
Done.


In [17]:
importances = best_model.get_feature_importance()
feature_names = best_model.feature_names_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("--- Feature Importance Analysis ---")
print(f"Total features: {len(feature_importance_df)}")

print("\nTop 10 Most Important Features:")
print(feature_importance_df.head(10))

print("\nBottom 10 Least Important Features:")
print(feature_importance_df.tail(10))

n_features_to_remove = 100
n_features_to_keep = len(feature_importance_df) - n_features_to_remove
top_features = feature_importance_df.head(n_features_to_keep)['feature'].tolist()

print(f"\nKeeping top {len(top_features)} features and removing bottom {n_features_to_remove}.")

X_train_top_features = X_train_proc[top_features]
X_test_top_features = X_test_proc.reindex(columns=top_features, fill_value=0) 

original_cat_features = set(CAT_FEATURES)
top_features_set = set(top_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

print(f"Original categorical features: {len(CAT_FEATURES)}")
print(f"Categorical features kept: {len(new_cat_features)}")

best_params_from_optuna = study.best_trial.params

final_params_new = best_params_from_optuna.copy()
final_params_new.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 50
})

print("\nRetraining model with top features...")
new_model = CatBoostClassifier(**final_params_new)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

print("\n--- Model Performance Comparison ---")
# 'final_f1' comes from cell 30 in your notebook
print(f"Original F1 score (all features): {final_f1:.4f}")
print(f"New F1 score (top {len(top_features)} features): {new_f1:.4f}")

print("\nNew Model Classification Report (Top Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))

--- Feature Importance Analysis ---
Total features: 136

Top 10 Most Important Features:
                        feature  importance
135  recovery_feasibility_score    6.169598
44              liab_x_multicar    5.522465
53               liab_prct_sqrt    3.782572
58     is_multi_vehicle_unclear    3.758837
45         liab_x_highrisk_site    3.383531
54                liab_prct_log    3.164801
52              liab_prct_cubed    3.108423
51            liab_prct_squared    3.107916
134           in_network_repair    2.934785
127                 recent_move    2.717762

Bottom 10 Least Important Features:
                   feature  importance
49       police_x_multicar         0.0
94            high_mileage         0.0
96       very_high_mileage         0.0
97        frequent_claimer         0.0
100  very_frequent_claimer         0.0
101           large_payout         0.0
66         evidence_strong         0.0
103           small_payout         0.0
104      very_large_payout         0.0


In [18]:
# Output module, from model_citizens.ipynb
real_test = pd.read_csv("data/Testing_TriGuard.csv")

X_real_test_proc = pre.transform(real_test)
X_real_test_proc = X_real_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)
real_pred_proba = best_model.predict_proba(X_real_test_proc)[:, 1]
real_pred_label = (real_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": real_pred_label
})

print(prediction.head())

Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
   claim_number  subrogation
0       3126034            0
1       7380142            1
2       4655051            0
3       6728725            1
4       9848460            1


In [19]:
prediction.to_csv("results/catboost_6034_prediction.csv", index=False)